In [58]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from bs4 import BeautifulSoup
import requests

In [59]:
url = 'https://www.imdb.com/chart/top'

## Fetch the webpage
r = requests.get(url)
print type(r)

<class 'requests.models.Response'>


In [60]:
r.status_code

200

In [61]:
info = r.text
print type(info)
print info[:500]

<type 'unicode'>



<!DOCTYPE html>
<html
    xmlns:og="http://ogp.me/ns#"
    xmlns:fb="http://www.facebook.com/2008/fbml">
    <head>
         
        <meta charset="utf-8">
        <meta http-equiv="X-UA-Compatible" content="IE=edge">

    
    
    

    
    
    

    <meta name="apple-itunes-app" content="app-id=342792525, app-argument=imdb:///?src=mdot">
            <style>
                body#styleguide-v2 {
                    background: no-repeat fixed center top #000;
                }
           


In [62]:
## soup is a DOM tree
soup = BeautifulSoup(r.text, 'html.parser')

In [63]:
print type(soup)

<class 'bs4.BeautifulSoup'>


In [64]:
TITLES = []
YEARS = []
RATINGS = []
URLS = []
ACTORS = []
DIRECTORS = []
DESCRIPTIONS = []
GENRES = []
BUDGET = []

In [65]:
table_inf = soup.find_all('tbody',{'class': 'lister-list'})
print len(table_inf)
print type(table_inf[0])
doc = table_inf[0]

1
<class 'bs4.element.Tag'>


In [66]:
rows = doc.find_all('tr')
print len(rows)

250


In [67]:
test_row = rows[0]
title_col = test_row.find('td', {'class': 'titleColumn'})
print title_col.a
print title_col.a.text

<a href="/title/tt0111161/?pf_rd_m=A2FGELUUNOQJNL&amp;pf_rd_p=e31d89dd-322d-4646-8962-327b42fe94b1&amp;pf_rd_r=SZQY7K56BRMXMKKB9TC7&amp;pf_rd_s=center-1&amp;pf_rd_t=15506&amp;pf_rd_i=top&amp;ref_=chttp_tt_1" title="Frank Darabont (dir.), Tim Robbins, Morgan Freeman">The Shawshank Redemption</a>
The Shawshank Redemption


In [68]:
def getAdditionalData(url):
    allData = {}
    r1 = requests.get(url)
    info_movie = r1.text
    soup_movie = BeautifulSoup(info_movie, 'html.parser')
    movie_desc = soup_movie.find('span',{'itemprop': 'description'})
    allData['desc']=movie_desc.text.strip()
    movie_genre = soup_movie.find_all('div',{'itemprop': 'genre'})
    result = []
    for th in movie_genre:
        result.extend(th.find_all('a'))
    genres_list = ''
    for ix in result:
        genres_list += (ix.text+', ')
    allData['genres'] = genres_list
    collection = soup_movie.find_all('div',{'class': 'txt-block'})
    fin_collection = ''
    for ix in collection:
        cur = ix.text.strip().split(':')
        if cur[0] == 'Budget':
            fin_collection = cur[1].split('(')[0].strip()
    allData['budget'] = fin_collection
    return allData

In [69]:
# count = 0
movie_root_url = 'https://www.imdb.com/'
for row in rows:
#     if count>1:
#         break
#     count += 1
    title_col = row.find('td', {'class': 'titleColumn'})
    rating_col = row.find('td', {'class': 'ratingColumn'})
    
    movie_title = title_col.a.text
    addnData = getAdditionalData(movie_root_url+title_col.a['href'])
    print addnData
#     year = title_col.span.text.split('(')[-1].split(')')[0]
    year = title_col.span.text[1:-1]
    rating = rating_col.strong.text
    movie_rel_url = title_col.a['href']
    movie_url = movie_root_url + movie_rel_url
    im_people = title_col.a['title']
    director = im_people.split(',')[0].split('(')[0].strip()
    actors = ','.join(im_people.split(',')[1:])
#     print movie_title
#     print year
#     print rating
#     print movie_url
#     print director
    actors = actors.strip()
#     print '--------------------'
    TITLES.append(movie_title.encode('utf-8'))
    YEARS.append(year.encode('utf-8'))
    URLS.append(movie_url.encode('utf-8'))
    RATINGS.append(rating.encode('utf-8'))
    ACTORS.append(actors.encode('utf-8'))
    DESCRIPTIONS.append(addnData['desc'].encode('utf-8'))
    BUDGET.append(addnData['budget'].encode('utf-8'))
    GENRES.append(addnData['genres'].encode('utf-8'))

{'genres': u' Drama, ', 'budget': u'$25,000,000', 'desc': u'The Shawshank Redemption is a highly-acclaimed movie starring Tim Robbins and Morgan Freeman. Andy Dufresne is convicted of the murder of his wife and her lover, and sentenced to life imprisonment at Shawshank prison. Life seems to have taken a turn for the worse, but fortunately Andy befriends some of the other inmates, in particular a character known only as Red. Over time Andy finds ways to live out life with relative ease as one can in a prison, leaving a message for all that while the body may be locked away in a cell, the spirit can never be truly imprisoned.'}
{'genres': u' Crime,  Drama, ', 'budget': u'$6,000,000', 'desc': u'When the aging head of a famous crime family decides to transfer his position to one of his subalterns, a series of unfortunate events start happening to the family, and a war begins between all the well-known families leading to insolence, deportation, murder and revenge, and ends with the favorab

{'genres': u' Adventure,  Drama,  Fantasy, ', 'budget': u'$94,000,000', 'desc': u"The continuing quest of Frodo and the Fellowship to destroy the One Ring. Frodo and Sam discover they are being followed by the mysterious Gollum. Aragorn, the Elf archer Legolas and Gimli the Dwarf encounter the besieged Rohan kingdom, whose once great King Theoden has fallen under Saruman's deadly spell."}
{'genres': u' Drama, ', 'budget': u'$4,400,000', 'desc': u'McMurphy has a criminal past and has once again gotten himself into trouble and is sentenced by the court. To escape labor duties in prison, McMurphy pleads insanity and is sent to a ward for the mentally unstable. Once here, McMurphy both endures and stands witness to the abuse and degradation of the oppressive Nurse Ratched, who gains superiority and power through the flaws of the other inmates. McMurphy and the other inmates band together to make a rebellious stance against the atrocious Nurse.'}
{'genres': u' Biography,  Crime,  Drama, ', 

{'genres': u' Animation,  Adventure,  Family,  Fantasy,  Mystery, ', 'budget': u'$19,000,000', 'desc': u"Chihiro and her parents are moving to a small Japanese town in the countryside, much to Chihiro's dismay. On the way to their new home, Chihiro's father makes a wrong turn and drives down a lonely one-lane road which dead-ends in front of a tunnel. Her parents decide to stop the car and explore the area. They go through the tunnel and find an abandoned amusement park on the other side, with its own little town. When her parents see a restaurant with great-smelling food but no staff, they decide to eat and pay later. However, Chihiro refuses to eat and decides to explore the theme park a bit more. She meets a boy named Haku who tells her that Chihiro and her parents are in danger, and they must leave immediately. She runs to the restaurant and finds that her parents have turned into pigs. In addition, the theme park turns out to be a town inhabited by demons, spirits, and evil gods. 

{'genres': u' Action,  Sci-Fi, ', 'budget': u'$102,000,000', 'desc': u"Over 10 years have passed since the first cyborg called The Terminator tried to kill Sarah Connor and her unborn son, John Connor. John Connor, the future leader of the human resistance, is now a healthy young boy. However another Terminator is sent back through time called the T-1000, which is more advanced and more powerful than its predecessor. The Mission: to kill John Connor when he's still a child. However, Sarah and John do not have to face this threat of a Terminator alone. Another Terminator is also sent back through time. The mission: to protect John and Sarah Connor at all costs. The battle for tomorrow has begun..."}
{'genres': u' Adventure,  Comedy,  Sci-Fi, ', 'budget': u'$19,000,000', 'desc': u'Marty McFly, a typical American teenager of the Eighties, is accidentally sent back to 1955 in a plutonium-powered DeLorean "time machine" invented by a slightly mad scientist. During his often hysterical, alwa

{'genres': u' Drama,  Film-Noir, ', 'budget': u'$1,752,000', 'desc': u"In Hollywood of the 50's, the obscure screenplay writer Joe Gillis is not able to sell his work to the studios, is full of debts and is thinking in returning to his hometown to work in an office. While trying to escape from his creditors, he has a flat tire and parks his car in a decadent mansion in Sunset Boulevard. He meets the owner and former silent-movie star Norma Desmond, who lives alone with her butler and driver Max Von Mayerling. Norma is demented and believes she will return to the cinema industry, and is protected and isolated from the world by Max, who was her director and husband in the past and still loves her. Norma proposes Joe to move to the mansion and help her in writing a screenplay for her comeback to the cinema, and the small-time writer becomes her lover and gigolo. When Joe falls in love for the young aspirant writer Betty Schaefer, Norma becomes jealous and completely insane and her madness

...

{'genres': u' Biography,  Drama,  History, ', 'budget': '', 'desc': u"The sufferings of a martyr, Jeanne D'Arc (1412-1431). Jeanne appears in court where Cauchon questions her and d'Estivet spits on her. She predicts her rescue, is taken to her cell, and judges forge evidence against her. In her cell, priests interrogate her and judges deny her the Mass. Threatened first in a torture chamber and then offered communion if she will recant, she refuses. At a cemetery, in front of a crowd, a priest and supporters urge her to recant; she does, and Cauchon announces her sentence. In her cell, she explains her change of mind and receives communion. In the courtyard at Rouen castle, she burns at the stake; the soldiers turn on the protesting crowd."}
{'genres': u' Drama,  Sci-Fi, ', 'budget': u'RUR\xa06,000,000', 'desc': u'In a small, unnamed country there is an area called the Zone. It is apparently inhabited by aliens and contains the Room, where in it is believed wishes are granted. The gov

{'genres': u' Drama,  Romance, ', 'budget': u'$2,500,000', 'desc': u'American tourist Jesse and French student Celine meet by chance on the train from Budapest to Vienna. Sensing that they are developing a connection, Jesse asks Celine to spend the day with him in Vienna, and she agrees. So they pass the time before his scheduled flight the next morning together. How do two perfect strangers connect so intimately over the course of a single day? What is that special thing that bonds two people so strongly? As their bond turns to love, what will happen to them the next morning when Jesse flies away?'}
{'genres': u' Drama,  Mystery,  Sci-Fi,  Thriller, ', 'budget': u'$150,000,000', 'desc': u"Thirty years after the events of the first film, a new blade runner, LAPD Officer K (Ryan Gosling), unearths a long-buried secret that has the potential to plunge what's left of society into chaos. K's discovery leads him on a quest to find Rick Deckard (Harrison Ford), a former LAPD blade runner who

{'genres': u' Drama,  History, ', 'budget': u'$800,000', 'desc': u"The Joad clan, introduced to the world in John Steinbeck's iconic novel, is looking for a better life in California. After their drought-ridden farm is seized by the bank, the family -- led by just-paroled son Tom -- loads up a truck and heads West. On the road, beset by hardships, the Joads meet dozens of other families making the same trek and holding onto the same dream. Once in California, however, the Joads soon realize that the promised land isn't quite what they hoped."}
{'genres': u' Adventure,  Drama,  History,  War, ', 'budget': u'$11,000,000', 'desc': u'In the Eighteenth Century, in a small village in Ireland, Redmond Barry is a young farm boy in love with his cousin Nora Brady. When Nora gets engaged to the British Captain John Quin, Barry challenges him to a duel of pistols. He wins and escapes to Dublin but is robbed on the road. Without an alternative, Barry joins the British Army to fight in the Seven Ye

{'genres': u' Crime,  Drama,  Film-Noir,  Thriller, ', 'budget': u'$829,000', 'desc': u"Mexican Narcotics officer Ramon Miguel 'Mike' Vargas has to interrupt his honeymoon on the Mexican-US border when an American building contractor is killed after someone places a bomb in his car. He's killed on the US side of the border but it's clear that the bomb was planted on the Mexican side. As a result, Vargas delays his return to Mexico City where he has been mounting a case against the Grandi family crime and narcotics syndicate. Police Captain Hank Quinlan is in charge on the US side and he soon has a suspect, a Mexican named Manolo Sanchez. Vargas is soon onto Quinlan and his Sergeant, Pete Menzies, when he catches them planting evidence to convict Sanchez. With his new American wife, Susie, safely tucked away in a hotel on the US side of the border - or so he thinks - he starts to review Quinlan's earlier cases. While concentrating on the corrupt policeman however, the Grandis have their

{'genres': u' Drama, ', 'budget': u'\xa31,162,000', 'desc': u'A man wanders out of the desert after a four year absence. His brother finds him, and together they return to L.A. to reunite the man with his young son. Soon after, he and the boy set out to locate the mother of the child, who left shortly after the man disappeared.'}
{'genres': u' Action,  Adventure,  Fantasy, ', 'budget': u'$140,000,000', 'desc': u"This swash-buckling tale follows the quest of Captain Jack Sparrow, a savvy pirate, and Will Turner, a resourceful blacksmith, as they search for Elizabeth Swann. Elizabeth, the daughter of the governor and the love of Will's life, has been kidnapped by the feared Captain Barbossa. Little do they know, but the fierce and clever Barbossa has been cursed. He, along with his large crew, are under an ancient curse, doomed for eternity to neither live, nor die. That is, unless a blood sacrifice is made."}
{'genres': u' Drama,  War, ', 'budget': u'$800,000', 'desc': u"A film commissi

In [70]:
print len(TITLES)

250


In [71]:
numpy_df = pd.DataFrame({'Title': TITLES, "Year": YEARS, 'Url': URLS, 'Rating': RATINGS, 'Actors':ACTORS, 'Description':DESCRIPTIONS, 'Genres':GENRES, 'Budget':BUDGET}, columns = ['Title', 'Year', 'Url', 'Rating', 'Actors', 'Description', 'Genres', 'Budget'])





In [72]:
numpy_df.head(n=4)

,Title,Year,Url,Rating,Actors,Description,Genres,Budget
0,The Shawshank Redemption,1994,https://www.imdb.com//title/tt0111161/?pf_rd_m...,9.2,"Tim Robbins, Morgan Freeman",The Shawshank Redemption is a highly-acclaimed...,"Drama,","$25,000,000"
1,The Godfather,1972,https://www.imdb.com//title/tt0068646/?pf_rd_m...,9.2,"Marlon Brando, Al Pacino",When the aging head of a famous crime family d...,"Crime, Drama,","$6,000,000"
2,The Godfather: Part II,1974,https://www.imdb.com//title/tt0071562/?pf_rd_m...,9.0,"Al Pacino, Robert De Niro",The continuing saga of the Corleone crime fami...,"Crime, Drama,","$13,000,000"
3,The Dark Knight,2008,https://www.imdb.com//title/tt0468569/?pf_rd_m...,9.0,"Christian Bale, Heath Ledger",Set within a year after the events of Batman B...,"Action, Crime, Drama, Thriller,","$185,000,000"


In [73]:
numpy_df.to_csv('./imdb.csv', index = False)

In [74]:
movie_data = pd.read_csv('imdb.csv')
movie_data.head(n=100)

,Title,Year,Url,Rating,Actors,Description,Genres,Budget
0,The Shawshank Redemption,1994,https://www.imdb.com//title/tt0111161/?pf_rd_m...,9.2,"Tim Robbins, Morgan Freeman",The Shawshank Redemption is a highly-acclaimed...,"Drama,","$25,000,000"
1,The Godfather,1972,https://www.imdb.com//title/tt0068646/?pf_rd_m...,9.2,"Marlon Brando, Al Pacino",When the aging head of a famous crime family d...,"Crime, Drama,","$6,000,000"
2,The Godfather: Part II,1974,https://www.imdb.com//title/tt0071562/?pf_rd_m...,9.0,"Al Pacino, Robert De Niro",The continuing saga of the Corleone crime fami...,"Crime, Drama,","$13,000,000"
3,The Dark Knight,2008,https://www.imdb.com//title/tt0468569/?pf_rd_m...,9.0,"Christian Bale, Heath Ledger",Set within a year after the events of Batman B...,"Action, Crime, Drama, Thriller,","$185,000,000"
4,12 Angry Men,1957,https://www.imdb.com//title/tt0050083/?pf_rd_m...,8.9,"Henry Fonda, Lee J. Cobb",The defense and the prosecution have rested an...,"Crime, Drama,","$350,000"
5,Schindler's List,1993,https://www.imdb.com//title/tt0108052/?pf_rd_m...,8.9,"Liam Neeson, Ralph Fiennes",Oskar Schindler is a vainglorious and greedy G...,"Biography, Drama, History,","$22,000,000"
6,The Lord of the Rings: The Return of the King,2003,https://www.imdb.com//title/tt0167260/?pf_rd_m...,8.9,"Elijah Wood, Viggo Mortensen",The final confrontation between the forces of ...,"Adventure, Drama, Fantasy,","$94,000,000"
7,Pulp Fiction,1994,https://www.imdb.com//title/tt0110912/?pf_rd_m...,8.9,"John Travolta, Uma Thurman",Pulp Fiction is a movie starring John Travolta...,"Crime, Drama,","$8,000,000"
8,"Il buono, il brutto, il cattivo",1966,https://www.imdb.com//title/tt0060196/?pf_rd_m...,8.8,"Clint Eastwood, Eli Wallach",Blondie (The Good) is a professional gunslinge...,"Western,","$1,200,000"
9,Fight Club,1999,https://www.imdb.com//title/tt0137523/?pf_rd_m...,8.8,"Brad Pitt, Edward Norton",A nameless first person narrator (Edward Norto...,"Drama,","$63,000,000"
